# Автодополнение текста: LSTM и Transformer

Этот ноутбук запускает полный пайплайн проекта:

1. Подготовка датасета (train / val / test) из `data/raw_dataset.txt`.
2. Обучение LSTM-модели автодополнения и замер метрики ROUGE.
3. Оценка трансформера `distilgpt2` из `Transformers` и замер метрики ROUGE + вывод примеров предсказаний.
4. Предсказание 20 продолжений с помощью двух моделей: авторской `lstm` и `distilgpt2` 


In [1]:
import os
import sys
from pathlib import Path

# Добавляем папку src в sys.path, чтобы можно было импортировать модули проекта
sys.path.append(os.path.join(os.getcwd(), "src"))

from data_utils import load_and_preprocess_text, train_val_test_split

DATA_DIR = Path("./data")
RAW_PATH = DATA_DIR / "raw_dataset.txt"

print("1/3: Подготовка датасета из raw_dataset.txt...")

if not RAW_PATH.exists():
    raise FileNotFoundError(
        f"Не найден файл с сырыми данными: {RAW_PATH}. Поместите raw_dataset.txt в папку data."
    )

splittedtext = load_and_preprocess_text(str(RAW_PATH))
train_tokens, val_tokens, test_tokens = train_val_test_split(splittedtext)

DATA_DIR.mkdir(parents=True, exist_ok=True)

with (DATA_DIR / "train_tokens.txt").open("w", encoding="utf-8") as f_train:
    for sent in train_tokens:
        f_train.write(" ".join(sent) + "\n")

with (DATA_DIR / "val_tokens.txt").open("w", encoding="utf-8") as f_val:
    for sent in val_tokens:
        f_val.write(" ".join(sent) + "\n")

with (DATA_DIR / "test_tokens.txt").open("w", encoding="utf-8") as f_test:
    for sent in test_tokens:
        f_test.write(" ".join(sent) + "\n")

print(
    f"Готово. Кол-во предложений: train={len(train_tokens)}, val={len(val_tokens)}, test={len(test_tokens)}"
)


e:\Yandex\Sprint 2. Text AutoComplete\lstm_text-autocomplete\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1/3: Подготовка датасета из raw_dataset.txt...
["- awww, that's a bummer. you shoulda got david carr of third day to do it.", "is upset that he can't update his facebook by texting it... and might cry as a result school today also. blah!", 'i dived many times for the ball. managed to save 50% the rest go out of bounds', 'my whole body feels itchy and like its on fire', "no, it's not behaving at all. i'm mad. why am i here? because i can't see you all over there."]
Готово. Кол-во предложений: train=1277731, val=159716, test=159717


In [2]:
import os
import sys

# Убеждаемся, что папка src есть в sys.path
sys.path.append(os.path.join(os.getcwd(), "src"))

from lstm_train import train_lstm_model

print("2/3: Обучение LSTM-модели и замер ROUGE на валидации...")
train_lstm_model()
print("Обучение LSTM завершено, модель сохранена в models/lstm_autocomplete.pt")



2/3: Обучение LSTM-модели и замер ROUGE на валидации...
Используемое устройство: cuda
Размер словаря: 89302
Эпоха: 1
ROUGE считаем на фиксированных 1000 примерах теста.
Эпоха 1/10 | train loss: 6.6025 | Test ROUGE-1: 0.1041 | Test ROUGE-2: 0.0087
Эпоха: 2
ROUGE считаем на фиксированных 1000 примерах теста.
Эпоха 2/10 | train loss: 5.9934 | Test ROUGE-1: 0.1094 | Test ROUGE-2: 0.0088
Эпоха: 3
ROUGE считаем на фиксированных 1000 примерах теста.
Эпоха 3/10 | train loss: 5.7098 | Test ROUGE-1: 0.1119 | Test ROUGE-2: 0.0085
Эпоха: 4
ROUGE считаем на фиксированных 1000 примерах теста.
Эпоха 4/10 | train loss: 5.5045 | Test ROUGE-1: 0.1050 | Test ROUGE-2: 0.0081
Эпоха: 5
ROUGE считаем на фиксированных 1000 примерах теста.
Эпоха 5/10 | train loss: 5.3298 | Test ROUGE-1: 0.1136 | Test ROUGE-2: 0.0125
Эпоха: 6
ROUGE считаем на фиксированных 1000 примерах теста.
Эпоха 6/10 | train loss: 5.1704 | Test ROUGE-1: 0.1134 | Test ROUGE-2: 0.0131
Эпоха: 7
ROUGE считаем на фиксированных 1000 примерах тест

In [1]:
import os
import sys

# Убеждаемся, что папка src есть в sys.path
sys.path.append(os.path.join(os.getcwd(), "src"))

from eval_transformer_pipeline import evaluate_distilgpt2_on_val

print("3/3: Оценка трансформера distilgpt2 на валидационной выборке...")
evaluate_distilgpt2_on_val()
print("Оценка distilgpt2 завершена.")



e:\Yandex\Sprint 2. Text AutoComplete\lstm_text-autocomplete\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3/3: Оценка трансформера distilgpt2 на валидационной выборке...


Device set to use cuda:0
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are no

Оценка модели distilgpt2 на тестовой выборке:
Всего сгенерировано: 917 примеров
Оценка ROUGE выполнена на: 917 примерах
ROUGE-1: 0.0720 | ROUGE-2: 0.0125

Примеры предсказаний:

=== Пример 1 ===
Префикс (3/4): just found out there was (and always has been) a shoprite in bklyn i could have transferred a long
Истинное продолжение (1/4): time ago. i mean come on, 7.65/hour!
Предсказанное продолжение:   list of items to the shoprite

=== Пример 2 ===
Префикс (3/4): i got micah to ride his 2nd roller coaster ever. so far he's ridden only wooden coasters - the
Истинное продолжение (1/4): grizzly + the hurler. next up: ricochet
Предсказанное продолжение:   only other coaster I've ridden is

=== Пример 3 ===
Префикс (3/4): just finished the beta of the home page... after 10:30pm and i realize i forgot to
Истинное продолжение (1/4): eat today! hungry... wonder what's open?
Предсказанное продолжение:   post it.

=== Пример 4 ===
Префикс (3/4): lots to do tomorrow... but now
Истинное продолжение 

In [4]:
import os
import sys

# Убеждаемся, что папка src есть в sys.path
sys.path.append(os.path.join(os.getcwd(), "src"))

from eval_difference import compare_distil_lstm_predictions

print("4/4: Сравнение предсказаний distilgpt2 и LSTM на одинаковых выражениях...")
compare_distil_lstm_predictions()
print("Сравнение завершено.")


4/4: Сравнение предсказаний distilgpt2 и LSTM на одинаковых выражениях...


Device set to use cuda:0
e:\Yandex\Sprint 2. Text AutoComplete\lstm_text-autocomplete\src\eval_difference.py:46: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = t

Сравнение предсказаний distilgpt2 и LSTM на 20 одинаковых выражениях:
=== Пример 1 ===
Префикс (3/4): yeah i know. i was fixing what u said.
Истинное продолжение (1/4): im testing 2
distilgpt2: 
LSTM:       lol i love

=== Пример 2 ===
Префикс (3/4): already got up it's #music time! wes montgomery - moã¯â¿â½a flor playing
Истинное продолжение (1/4): now. good morning, jazz!
distilgpt2: in the background.
LSTM:       <unk> <unk> <unk> <unk>

=== Пример 3 ===
Префикс (3/4): i've been there. the only place i have flown out of since moving up north. really pretty
Истинное продолжение (1/4): area for flying, but very expensive
distilgpt2: much the only thing i have
LSTM:       good thing i have to go

=== Пример 4 ===
Префикс (3/4): whoever invented the subject business management will get my
Истинное продолжение (1/4): foot up there ass.
distilgpt2: attention.”
LSTM:       <unk> <unk> <unk> <unk>

=== Пример 5 ===
Префикс (3/4): it's late right now, no one is around, no one is online, i'm 

1. **LSTM‑модель.** Обучение прошло 10 эпох на GPU. Метрики ROUGE низкие и
   стагнируют: ROUGE‑1 колеблется около `0.10–0.114`, ROUGE‑2 около
   `0.008–0.015` (лучшие значения: ROUGE‑1 ≈ 0.1136 на 5‑й эпохе,
   ROUGE‑2 ≈ 0.0145 на 8‑й).
2. **Transformer (distilgpt2).** Метрики ROUGE на 917 примерах: **ROUGE-1: 0.0720 | ROUGE-2: 0.0125**.
   Результаты ниже, чем у LSTM по ROUGE-1, но сопоставимы по ROUGE-2.
3. LSTM показывает лучшие результаты по ROUGE-1 (0.1136 vs 0.0720), что указывает на лучшее
   совпадение отдельных слов. По ROUGE-2 результаты близки (0.0145 vs 0.0125).

Качественный анализ генераций:
   - LSTM часто генерирует последовательности с множеством `<unk>` токенов
     (примеры 2, 4, 5, 12, 13, 16). В редких случаях генерирует осмысленные продолжения
     (пример 19: "just got home. about to go for a walk with my").
   - distilgpt2: генерирует более правильный текст без `<unk>`,
     но часто даёт пустые или очень короткие продолжения (примеры 1, 8, 11).
     Продолжения семантически не всегда соответствуют контексту, но выглядят
     естественно (пример 3: "much the only thing i have" вместо "area for
     flying, but very expensive").
   - Обе модели редко точно предсказывают истинное
     продолжение, что согласуется с низкими метриками ROUGE. 
     distilgpt2 демонстрирует лучшую корректность, но LSTM иногда лучше улавливает семантику.